In [126]:
import queue
import itertools 

In [ ]:
grocery_map = [
        [3,3,3,4,4,4],
        [0,0,0,0,0,0],
        [5,0,1,2,0,6],    
        [5,0,1,2,0,6],
        [5,0,1,2,0,6],
        [0,0,0,0,0,0]
        ]


In [55]:
#from https://techwithtim.net/tutorials/breadth-first-search/

def findPathLength(floorplan, source, target, verbose = False):
    """Given a grid (floorplan) computes the shortest path between source and target"""
    
    def find_lat(floorplan, source): 
        """Finds the y coordinate of the source to use later"""
        for n_i, i in enumerate(floorplan):
            for j in i:
                if j == source:
                    return n_i
                
    def map_to_maze(floorplan, source, target):
        """
        Just changes elements of non interest into '#' barriers
        """
        source = source + 'm'; target = target + 'm'
        new_map = []
        for n_i, row in enumerate(floorplan):
            new_row = []
            for elt in row:
                if elt == " " or elt == source or elt == target:
                    new_row.append(elt[0])

                else:
                    new_row.append('#')
            new_map.append(new_row)

        return new_map

    def printMaze(maze, source, path=""):
        lat = find_lat(maze, source)
        for x, pos in enumerate(maze[lat]):
            if pos == source:
                start = x

        i = start
        j = lat
        pos = set()
        for move in path:
            if move == "L":
                i -= 1
            elif move == "R":
                i += 1
            elif move == "U":
                j -= 1
            elif move == "D":
                j += 1
            pos.add((j, i))

        for j, row in enumerate(maze):
            for i, col in enumerate(row):
                if (j, i) in pos:
                    print("+ ", end="")
                else:
                    print(col + " ", end="")
            print()

    def valid(maze, moves, source):
        lat = find_lat(maze, source)
        for x, pos in enumerate(maze[lat]):
            if pos == source:
                start = x

        i = start
        j = lat
        for move in moves:
            if move == "L":
                i -= 1
            elif move == "R":
                i += 1
            elif move == "U":
                j -= 1
            elif move == "D":
                j += 1

            if not(0 <= i < len(maze[0]) and 0 <= j < len(maze)):
                return False

            elif (maze[j][i] == "#"):
                return False

        return True, len(moves)


    def findEnd(maze, moves, source, target, verbose):
        lat = find_lat(maze, source)
        for x, pos in enumerate(maze[lat]):
            if pos == source:
                start = x

        i = start
        j = lat
        for move in moves:
            if move == "L":
                i -= 1
            elif move == "R":
                i += 1
            elif move == "U":
                j -= 1
            elif move == "D":
                j += 1

        if maze[j][i] == target:
            if verbose == True:
                print("Directions: " + moves)
                print("Length: " , len(moves) - 1)
                printMaze(maze, source, moves)
            return True, len(moves)

        return False


    # MAIN ALGORITHM

    nums = queue.Queue()
    nums.put("")
    add = ""
    maze = map_to_maze(floorplan, source, target)

    while not findEnd(maze, add, source, target, verbose): 
        add = nums.get()
        #print(add)
        for j in ["L", "R", "U", "D"]:
            put = add + j
            if valid(maze, put, source):
                nums.put(put)
    return len(add) - 1 
    
# l = findPathLength(vons, "6m", "1m", verbose = True)
# l

In [3]:
grocery_map = [
        ["3","3m","3","4","4m","4"],
        [" "," "," ", " "," ", " "],
        ["5"," ","1","2"," ","6"],    
       ["5m"," ","1m","2m"," ","6m"],
        ["5"," ","1","2"," ","6"],
        [" "," "," "," "," "," "]
        ]

grocery_map2 = [
        [" "," "," "," "," ","X"],
        [" "," "," "," "," "," "],
        [" "," "," "," "," ","#"],    
       [" "," "," "," "," "," "],
        [" "," "," "," "," "," "],
        ["O"," "," "," "," "," "]
        ]

In [38]:
c = map_to_maze(grocery_map, '3m', '5m')
c

[['#', '3', '#', '#', '#', '#'],
 [' ', ' ', ' ', ' ', ' ', ' '],
 ['#', ' ', '#', '#', ' ', '#'],
 ['5', ' ', '#', '#', ' ', '#'],
 ['#', ' ', '#', '#', ' ', '#'],
 [' ', ' ', ' ', ' ', ' ', ' ']]

In [77]:
def make_paths(floorplan):
    """
    Finds all the sources and targets pairs, which are the middles of isles
    """
    isle_centers = []
    #won't need to do this part if I can just get isle ID's from grocery isle/item dictionary 
    for row in floorplan:
        for elt in row:
            try: 
                if type(int(elt[0])) == int:
                    isle_centers.append(elt[0])
            except ValueError:
                continue
        
    isle_centers = list(dict.fromkeys(isle_centers))
    #this is where I make unique pairs from the isle centers but I feel like it's a tad inefficent            
    sets = [{a, b} for a in isle_centers  
                   for b in isle_centers if a != b] 
    isle_pairs = []
    [isle_pairs.append(i) for i in sets if i not in isle_pairs]  
    isle_pairs = [tuple(i) for i in isle_pairs]
    
    distances = dict.fromkeys(isle_pairs)
    for pair in isle_pairs:
        source = pair[0]; target = pair[1]
        distances[pair] = findPathLength(floorplan, source, target)
      
    return distances

In [79]:
paths = make_paths(grocery_map)
paths

{('3', '4'): 4,
 ('3', '5'): 3,
 ('3', '1'): 3,
 ('3', '2'): 6,
 ('3', '6'): 6,
 ('5', '4'): 6,
 ('1', '4'): 6,
 ('2', '4'): 3,
 ('4', '6'): 3,
 ('1', '5'): 1,
 ('5', '2'): 8,
 ('5', '6'): 8,
 ('1', '2'): 0,
 ('1', '6'): 8,
 ('2', '6'): 1}

In [82]:
isles = {1:['pizza', 'frozen vegetables', 'frozen fruit'], 
             2:['dumplings', 'chicken tenders', 'hot pockets'],
             3:['milk', 'yogurt', 'cheese'], 
             4:['chicken','beef','pork'],
             5:['bread', 'bagels', 'muffins'],
             6:['bananas', 'apples', 'oranges']
            }

groclist = ['pizza', 'frozen vegetables', 'beef', 'bread', 'muffins', 'apples']

target_isles = []
for item in groclist: 
    for k, v in isles.items():
        if item in v:
            target_isles.append(k)
        
target_isles = list(dict.fromkeys(target_isles))
target_isles

[1, 4, 5, 6]

create edges from target isles, then look them up in the paths dictionary (account for order invariance (a,b) = (b,a))

In [163]:
def makeShortestPath(grocery_list):
    """
    Displays shortest path to traverse on grid visiting each location once
    So reduces a (size of isle list)Choose(2) sized list of edges into (size of isle list) - 1 sized list.
    """
    
    target_isles = []
    for item in grocery_list: 
        for k, v in isles.items():
            if item in v:
                target_isles.append(str(k))
   
    target_isles = list(dict.fromkeys(target_isles))
    print(target_isles)
    
    sets = [{a, b} for a in target_isles  
                   for b in target_isles if a != b] 
    isle_pairs = []
    [isle_pairs.append(i) for i in sets if i not in isle_pairs]  
    isle_pairs = [tuple(i) for i in isle_pairs]
    
    dpaths = dict.fromkeys(isle_pairs)
    for i in isle_pairs:
        if i in paths.keys() or (i[1], i[0]) in paths.keys():
            #try except block may not be necessary, but order matters in tuples
            try:
                dpaths[i] = paths[i]
            except KeyError:
                dpaths[(i[1], i[0])] = paths[(i[1], i[0])]

            # also might not need dpaths, since could just lookup weights from main dict
    possible_paths = list(itertools.combinations(isle_pairs, len(target_isles)-1)) 
    good_paths= []
    for path in possible_paths:
        if path[0] == 
    return dpaths

Now want to find the minimum weighted path from dpaths

In [164]:
x = makeShortestPath(groclist)
x


['1', '4', '5', '6']


{('1', '4'): 6,
 ('1', '5'): 1,
 ('1', '6'): 8,
 ('5', '4'): 6,
 ('4', '6'): 3,
 ('5', '6'): 8}

In [90]:
paths[('3', '4')]

4

In [162]:
v = [1,2,3,4,5,6]

for i in v:
    if i%2 ==0:
        print(i)
        v.remove(i)

v

2
4
6


[1, 3, 5]

In [150]:
for n, i in enumerate([1,2,3,4]):
    print(n)

0
1
2
3
